# Implementing the Transformer

Reference: [Implementation_Tutorial](Transformer_Implementation_Tutorial.ipynb)

In [1]:
from torch import nn 
import torch

## EmbeddingWithPositionalEncoding

In [ ]:
class EmbeddingWithPositionalEncoding(nn.Module):
    def __init__(self, num_embeddings: int, d_embed: int, d_model: int):
        super().__init__()
        self.d_model = d_model
        self.d_embed = d_embed
        self.embedding = nn.Embedding(
            num_embeddings=num_embeddings,
            embedding_dim=d_embed
        )

    @staticmethod
    def create_positional_encoding()


##